In [19]:
from slim_gsgp_lib.main_slim import slim
from slim_gsgp_lib.main_gsgp import gsgp 
from slim_gsgp_lib.main_gp import gp
from slim_gsgp_lib.utils.utils import train_test_split
from slim_gsgp_lib.evaluators.fitness_functions import rmse
from slim_gsgp_lib.utils.callbacks import EarlyStopping
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import time
import os
from tqdm import tqdm
from functions.test_funcs import mape, nrmse, r_squared, mae, standardized_rmse


from functions.test_algorithms import *
from functions.random_search import * 
from slim_gsgp_lib.datasets.data_loader import *
import pickle

datasets = [globals()[i] for i in globals() if 'load' in i][2:]

Timeout 1 (19) - Iteration 0 - Pattern: parkinson_updrs - Seed: 2477
p_inflate, max_depth, init_depth, prob_const, prob_replace, p_prune, p_xo, p_struct_xo, struct_mutation, tournament_size, ms_lower, ms_upper)
   0.7        19          7          0.3          0            0      0         0           False             2               0         0

In [20]:
for dataset_loader in datasets:
    X,y = dataset_loader()
    print(f"Dataset: {dataset_loader.__name__}")
    print(f"Shape: {X.shape}")

Dataset: load_resid_build_sale_price
Shape: torch.Size([372, 107])
Dataset: load_istanbul
Shape: torch.Size([536, 7])
Dataset: load_airfoil
Shape: torch.Size([1503, 5])
Dataset: load_bike_sharing
Shape: torch.Size([731, 13])
Dataset: load_boston
Shape: torch.Size([506, 13])
Dataset: load_breast_cancer
Shape: torch.Size([569, 30])
Dataset: load_concrete_slump
Shape: torch.Size([103, 7])
Dataset: load_concrete_strength
Shape: torch.Size([1005, 8])
Dataset: load_diabetes
Shape: torch.Size([442, 10])
Dataset: load_efficiency_heating
Shape: torch.Size([768, 8])
Dataset: load_efficiency_cooling
Shape: torch.Size([768, 8])
Dataset: load_forest_fires
Shape: torch.Size([513, 43])
Dataset: load_parkinson_updrs
Shape: torch.Size([5875, 19])
Dataset: load_ld50
Shape: torch.Size([234, 626])
Dataset: load_ppb
Shape: torch.Size([131, 626])
Dataset: load_bioav
Shape: torch.Size([358, 241])


In [2]:
X,y = datasets[12]()

# Scale
scaler = MinMaxScaler()
X = torch.tensor(scaler.fit_transform(X))
y = torch.tensor(scaler.fit_transform(y.reshape(-1,1)).reshape(-1))

X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=1-0.7, seed=3441)

earlystopping = EarlyStopping(patience=30)

start = time.time()
final_tree = slim(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, slim_version='SLIM+SIG2', pop_size=100, n_iter=250, seed=3441,
                      reconstruct=True, p_inflate=0.4, max_depth=20, init_depth=6, prob_const=0.3, prob_replace=0, p_prune=0, 
                      p_xo=0.1, p_struct_xo=1, struct_mutation=True, tournament_size=3, ms_lower=0, ms_upper=0, callbacks=[earlystopping],)
end = time.time()

c:\Users\Mateus\Documents\Tese\Codigo\SLIM-TEST-LITE\slim_gsgp_lib\main_slim.py:192: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|     dataset     |    iteration    |    train_fit    |     test_fit    |       time      |      nodes      |    diversity    |     avg_size    |     avg_fit     |     std_fit     |     avg_gpd0    |    avg_depth    |
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|    dataset_1    |        0        |      0.307      |      0.300      |      0.177      |        3        |    423464.620   |      1.000      |     3337.819    |    31538.664    |      3.090      |      3.090      |
|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-

**SLIM*SIG2**
| Parameter       |   Scaled |   Unscaled |   Difference |
|:----------------|---------:|-----------:|-------------:|
| p_inflate       |     0.70 |       0.50 |         0.20 |
| max_depth       |    23.00 |      20.00 |         3.00 |
| init_depth      |     8.00 |       5.00 |         3.00 |
| prob_const      |     0.10 |       0.05 |         0.05 |
| tournament_size |     2.00 |       3.00 |        -1.00 |
| ms_upper        |     1.00 |       0.00 |         1.00 |

In [17]:
with open('params/bike_sharing.pkl', 'rb') as f:
    arguments_unscaled = pickle.load(f)['SLIM*SIG2']
    
arguments_unscaled['init_depth'] = 6
arguments_unscaled['max_depth'] = 23
print(arguments_unscaled)

X,y = datasets[3]()
results_tuple = test_slim(X,y, algorithm='SLIM*SIG2', n_iter=250, pop_size=100, iterations=20, p_train=0.7, show_progress=True, log=False,
          scale=True, args_dict=arguments_unscaled)

{'p_inflate': 0.5, 'max_depth': 23, 'init_depth': 6, 'prob_const': 0.05, 'prob_replace': 0, 'p_prune': 0, 'p_xo': 0, 'p_struct_xo': 0, 'struct_mutation': False, 'tournament_size': 3, 'ms_lower': 0, 'ms_upper': 0}


100%|██████████| 20/20 [06:32<00:00, 19.62s/it]
